In [1]:
!pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 

In [2]:
import transformers
import re
import os, os.path
import fnmatch
import json
import collections
import glob
import json

import matplotlib.pyplot as plt
import numpy as np
# import modules for web scrapping
import requests
from bs4 import BeautifulSoup

import pandas as pd

In [3]:
persons = ["Mahatma_Gandhi","Nelson_Mandela","Abraham_Lincoln","Vladimir_Lenin","Jawaharlal_Nehru", "Nikola_Tesla","Albert_Einstein",\
          "Marie_Curie","Stephen_Hawking","Elon_Musk","Warren_Buffett","Jack_Ma","Oprah_Winfrey","Sachin_Tendulkar",\
          "Lionel_Messi","Milkha_Singh"]

df =  pd.DataFrame(columns = ['title', 'text'])

def FetchParagraphBetweenIds(id1,id2):
    hElem = soup.find("span", {'id': id1})
    endElem = soup.find('span', {'id': id2})
    cntns = list(soup.find_all())

    my_lst = []
    inBetween = False
    for tag in cntns:
        if tag == hElem:
            inBetween = True
        if inBetween == True and tag.name == 'p':
            my_lst.append(tag.get_text())
        if tag == endElem:
            inBetween = False
            break
    return "".join(my_lst)

for p in persons:
    url = f'https://en.wikipedia.org/wiki/{p}'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)
    #print(response.status_code)
    soup = BeautifulSoup(response.content, 'html.parser')

    desired_ids = []
    for link in soup.find_all('span', attrs={'class':'mw-headline'}):
        if link.get('id') is not None: 
            desired_ids.append(link.get('id'))

    id_to_paragraph = {}
    for i in range(len(desired_ids)-1):
        id_to_paragraph[desired_ids[i]] = FetchParagraphBetweenIds(desired_ids[i],desired_ids[i+1])
    id_to_paragraph_main = {}
    for k,v in id_to_paragraph.items():
        if len(v)>0:
            id_to_paragraph_main[k] = v
    df_new_row = pd.DataFrame(list(id_to_paragraph_main.items()), columns = ['title', 'text'])
    df = pd.concat([df, df_new_row])

df.reset_index(inplace=True)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')

model_checkpoint = "/content/drive/My Drive/title-generation/"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)   # we need this
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

<ipython-input-6-faaac16489c9>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [7]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

# dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

# ### convert to Huggingface dataset
# hg_dataset = Dataset(pa.Table.from_pandas(df))

## Calculate Rouge Score without finetuning

In [8]:
from sklearn.model_selection import train_test_split
training_dataset, test_dataset = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)
dataset_train = ds.dataset(pa.Table.from_pandas(training_dataset).to_batches())
dataset_test = ds.dataset(pa.Table.from_pandas(test_dataset).to_batches())

### convert to Huggingface dataset
hg_dataset_train = Dataset(pa.Table.from_pandas(training_dataset))
hg_dataset_test = Dataset(pa.Table.from_pandas(test_dataset))



In [9]:
def get_title(text):
    inputs = ["summarize: " + text]
    inputs = tokenizer(inputs, max_length=768, truncation=True, return_tensors="pt")
    output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
    decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]
    return predicted_title

def evaluate_baseline(dataset, metric):
    pred = [get_title(text) for text in dataset["text"]]
    return metric.compute(predictions=pred, references=dataset["title"], use_stemmer=True)

score = evaluate_baseline(test_dataset, metric)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)
rouge_dict


{'rouge1': 11.9, 'rouge2': 4.13, 'rougeL': 11.02, 'rougeLsum': 10.95}

In [9]:
import string
prefix = "summarize: "
max_input_length = 512
max_target_length = 64

def clean_text(text):
    sentences = nltk.sent_tokenize(text.strip())
    sentences_cleaned = [s for sent in sentences for s in sent.split("\n")]
    sentences_cleaned_no_titles = [sent for sent in sentences_cleaned
                                 if len(sent) > 0 and
                                 sent[-1] in string.punctuation]
    text_cleaned = "\n".join(sentences_cleaned_no_titles)
    return text_cleaned

def preprocess_data(examples):
    texts_cleaned = [clean_text(text) for text in examples["text"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["title"], max_length=max_target_length, 
                            truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
# tokenized_datasets = df.map(preprocess_data,batched=True)
# tokenized_datasets = preprocess_data(hg_dataset_train)
tokenized_datasets_train = hg_dataset_train.map(preprocess_data, batched=True)
tokenized_datasets_test = hg_dataset_test.map(preprocess_data, batched=True)

Map:   0%|          | 0/404 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainingArguments, Seq2SeqTrainer

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [16]:
batch_size = 4
model_name = model_checkpoint
args = Seq2SeqTrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=6,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=True,
)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [18]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
# Start TensorBoard before training to monitor it in progress
# %load_ext tensorboard
# %tensorboard --logdir '{model_dir}'/runs

In [20]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.960379,19.078600,9.372300,18.467300,18.211900,8.257400
2,No log,1.829504,23.576400,10.867600,22.443800,22.159100,7.792100
3,No log,1.786162,23.751800,10.041100,22.482300,22.301100,9.524800
4,No log,1.783755,24.167900,10.481100,23.241000,23.130200,9.227700
5,1.541300,1.796158,26.026200,10.602600,25.313000,24.980400,9.485100
6,1.541300,1.786058,25.830400,10.917400,24.969700,24.775400,8.960400


TrainOutput(global_step=606, training_loss=1.441908858396826, metrics={'train_runtime': 307.09, 'train_samples_per_second': 7.893, 'train_steps_per_second': 1.973, 'total_flos': 1442687847505920.0, 'train_loss': 1.441908858396826, 'epoch': 6.0})

In [ ]:
trainer.save_model('t5_fine_tuned')
